# Overview: Brucella_australia project

We're interested in building a phylogeny from a collection of reads that were sent from two separeate groups: one group from Germany, and another group from Australia.  

Whenever the Foster lab receives samples, whatever naming convention was provided by the sender is then modified by our lab - we label things numerically, thus the following SampleIDs are assigned by our lab. These may be labeled as something else by the sender.  


We received DNA from these two labs and labeled thems as follows. Our list includes:    

> __From Germany__  
_Golfer Scholz and Julia Reihm Germany_  
* Sample0436 - Sample 0448  

>__From Australia__  
_Pat Blackall_  
* Sample0449 - Sample0461


# Library prep and notes about samples

All DNA samples listed above were subject to the same library prep methods, however these preps were performed on separate days by two individuals (Daphne Welker and Devon O'Rourke). 

We followed previously [published methods] [link]. Samples were sequenced at the Hubbard Genome Center at the University of New Hampshire on an Illumina HiSeq2500. 250 bp PE sequencing was performed.  
[link]:http://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0128036

Sample0461 failed to produce any library, while Sample0453 and Sample0454 were rerun in a separate sequencing reaction due to low yields from the first run.  Sample0453 was (for an unknown reason) sequenced twice on that second run, with one sample being sequence on each of the two Illumina lanes. Initial analyses were conducted (described below) on both samples, named Sample0453L001 and Sample0453L002 to distinguish between the two lanes. Subsequent analyses confirmed that those two libraries were indeed from the same sample, and Sample0453L001 was used in final phylogenic anlayses.  


# Preliminary sequencing analyses
## Sample statistics from the genome center
Sample statistics are available for both the [initial] [link2] and [resequencing] [link3] runs. Note that samples 0453, 0454, and 0461 were discared from initial run for low coverage.  
[link2]:http://cobb.unh.edu/150424_MTB_Bsuis_DemuxStats.html
[link3] :

## FastQC analyses
FastQC was run on raw data for two samples and confirmed high quality yeild as suggeted by the above sample statistics: per base sequence quality remained high throughout read position and per sequence quality scores remained skewed well above a Phred of 30. Repeat nucleotides were focues only at the first 15 bp of the read, indicative of the need for adapter trimming (performed in our bioinforamtic analyses downstream). 

### minor note on transferring files:  

File existed preliminarily on the Hubbard "cobb" server, and were moved to our local "pseudomonas" server for initial bioinformatic work. These products were then moved to another server, "inquiry"for tree building. In each case, the program 'rsync' was used to copy files. See the [manual] [link4] for more information. The basic syntax for this is as follows:
[link4]:http://linuxcommand.org/man_pages/rsync1.html 

# NASP - alignment, variant calling, and SNP Matrix creation

The first major portion of our bioinformatic pipeline consists of a suite of tools implemented through [NASP] [link5]. The suite of tools called in a full workflow consist of three major processes:  

> - aligning reads to a reference - by default we use BWA  
> - identifying SNPs - by default with GATK  
> - creating a SNP matrix that can be used to compute branch lengths with external phylogeny-building software  

[link5]:http://tgennorth.github.io/NASP/index.html


## NASP beginnings - using the whole toolbox

Initial analyses utilized the entire suite of options in the NASP pipeline:  

> - a reference fasta was provided (must end in '.fasta')
> - duplicate regions were checked and skipped for SNPs for such regions
> - no system manager was used (Torgue or PBS weren't installed on 'pseudomonas')
> - exteranl genomes were included (all genomes must end in '.fasta')
> - read files were included (may be compressed in '.fastq.gz' format)
> - no pre-aligned SAM/BAM files were included
> - no pre-called VCF files were included
> - no low-quality positions failing coverage (or masked) are included in master_matrix file
> - minimum coverage threshold = 10
> - minimum acceptable proportion of reads matching call made by SNP caller = 0.9 (90%)

The program was executed in a tmux window and command prompts were entered with the appropriate paths to reference genomes (reference fasta), assemblies (external genomes), and sequencing samples (read files).  


## First NASP run

### NASP Input

The initial analysis conducted included all read files and broad representative set of external assemblies.  

The initial reference - **B. suis ATCC 23445** - was selected because we knew apriori that some of these samples were derived from a B. suis isolate.  

External references (assemblies) were used to represent a phylogenetically diverse suite of Brucellae. These included:  
    * Brucella_melitensis_bv_2_str_639
    * Brucella_melitensis_biovar_abortus_2308
    * Brucella_neotomae_5K33
    * Brucella_pinnipedialis_B294
    * Brucella_ceti_B194
    * Brucella_canis_ATCC_23365
    * Brucella_microti_CCM_4915
    * Brucella_sp_NF_2653
    * Brucella_sp_8313
    * Brucella_sp_BO2
    * Brucella_sp_BO1
    * Ochrobactrum_anthropi_ATCC_49188  

Samples 0436 - 0460 were included in this first run. 

### NASP Output

While there are many output files created throughout the pipeline (aligned .bam files, indel-called .vcf files, nucmer-aligned delta files, 1-1 position alignment frankenfastas... and more!), the key outputs of a successful run include two groups of two files that you'll want to delve into initially:  

> - The "sample_stats.tsv" and "general_stats.tsv" files:  
> - These include information about the percent of genomes aligned among your samples, the number of SNPs, and much more. This is a good quick check to determine how successful the read mapping was, and to what extent of those reads may be informative in terms of SNPs.  

Our initial run suggested that there was a clear split among the majority of the German and Australian samples. All of the German samples were mapping a far higher percentage to the reference than most Australian samples. This suggested that perhaps these Australian samples were not as related to the reference genome as the German ones. In addition, samples __0452, 0455, and 0457__ had a paltry mapping percentage, likely indicating that these were not actually Brucella samples. To confirm this, the first 100 reads of each sample were blasted through NCBI's 'nr' database. The output suggested that all of these samples are likely from some Acinetobacter species(s). Jeff Foster suggested that it may have been an error on the part of the German lab sending the wrong samples. These three samples were not used in subsequent analyses in future NASP runs.  

Because those three samples had such low percentage of reads mapped, there was no reason to continue with this initial analysis. The second set of files useful in determining the evolutionary relationships - that is, useful for building trees - include two files:

> * bestsnp.fasta
> * master.tsv  

However, these files require a relatively high proportion of reads to map across all samples. Because those three samples had so few reads mapped, the resulting "bestsnp.fasta" file was of no use in building the tree with this initial run. __Therefore, we discarded those three samples (0452, 0455, and 0457) and reran the entire pipeline as described above without those three samples. The following output and analysis excludes those samples__:  

The output files from the resulting NASP run were transferred to another server "inquiry.unh.edu" to make estimates of evolutionary relatedness using a maximum likelihood approach.

### RAxML Input
The Randomized Accelerated Maximum Likelihood ([RAxML] [link6]) program used here is not the only option to build trees, but it certainly a fast one. We use RAxML for all initial analyses following the NASP pipeline as it provides quick access to information to build trees typically within less than an hour. See [this site] [link7] for a gentle introduction explaining what a few of these commands listed below are doing.  

However, before you can run RAxML there's a weird quirk due to the formatting by NASP's "bestsnp.fasta" file that first needs to get fixed. Essentially RAxML doesn't like the headers in the .fasta file to contain the "::GATK...etc" text that is output from NASP. In addition, the reference is just labeled "reference" but we're going to attach a bit more information to that subject. To do this I created a file called "replace.sed" with the following info:

[link6]:http://sco.h-its.org/exelixis/web/software/raxml/
[link7]:http://evomics.org/learning/phylogenetics/raxml/

Notice above that we're just replacing the headers with slightly renamed versions of the same thing. Next up is to run the following command to substitute necessary headers:

Once that file is renamed, there are two options to running RAxML:

- Version_A is much faster but includes much less information (it doesn't include the detailed 'master.tsv' file at all)
- Version_B is slower but includes much more information (it includes both the "bestsnp.fasta" and the "master.tsv" file.  

__Version_A commands used include:__  
[-s] input fasta  
[-n] output filename  
[-m] substitution model  
[-f a] specifies a certain algorithm to run (in this case, this runs conducts a rapid bootstrap analysis)   
[-x] random number seed to initiate heuristic search. just know that if you enter the same number, you get the same ouput every time  
[-N autoMRE] number of replicates: we're letting RAxML decide with this specific 'autoMRE' option  
[-p] parsiomony inference: what you need to start building a tree  


__Version_B commands__
Kevin Drees developed a custom perl script to include the renamed "bestsnp.fasta" and "master.tsv" files while running custom RAxML settings. Ensure that you are running Python 3 and have loaded mpich3 (see notes on loading modules in "inquiry" server in script text below).  

To run this script:

### RAxML output and processing:

RAxML will create a series of output files. Only a single file is necessary to building a phylogeny in a tree-viewer program: it's the one listed as "__...bipartitions.nameoffasta__". Files were transferred from "inquiry" and on to a local machine.  

Next, install [FigTree] [link7] or other tree-viewer. Note that if you're going to install FigTree, you'll need to rename the file extension from RAxML from a '.out' file to a ',tre' extension.

#### First run output insights:

The resulting tree topology confirmed a few expectations:  

1. The German samples provided a broad representation of the variouis B. suis biovars. However, these samples were sent to represent such a (relatively) narrow clade, our initially broad external reference list did not allow further discrimination among the B. suis biovars and our samples.

2. The Australian samples generally clustered to the two rodent Brucella representatives which we would have guessed given that they were extracted from rodent hosts. There was a single sample, __0450__ which did not cluster to either of those two major clades, but this was also expected as it was extracted from a pig host; sure enough, that sample clustered to a B. suis clade.  

To better descriminate among the existing samples, further NASP pipelines were run to better understand:  

1. An almost identical run to what was describe above, but removing the outgroup of "Ochrobactrum" in our external references. Instead, we'll intend on rooting our tree with the most distantly related Brucella among our samples: B. sp B01 and B. sp B02.  

2. The relationship of the German and one australian B. suis - associated samples. Specifically samples __0436 - 0440, and 0450__.  

3. The relationship of the Australian samples mapping to the rodent-associated Brucella.  

These analyses are described below.
[link7]:http://tree.bio.ed.ac.uk/software/figtree/

## Refined NASP run

While the tree topology will be improved in subsequent analyses (see below) for the distinct clades in which our species are clustering, a first attempt at improving branch lengths of the existing tree is done by increasing the number of common loci between all of our samples and our references. The inclusing of Ochrobactrum reduces the likelihood of common loci because it so much more diverged to all other genomes in our analysis (reads and references alike); therefore by its removal we can improve the likelihood that we'll find more SNPs in our output.  

Because we're running the pipeline using the same B. suis reference (B. suis ATCC 23445), the process can begin by using the .vcf files produced in the previous pipeline instead of rerunning BWA and GATK to create those files. We rerun the pipeline while exlcuding that one single Ochrobactrum genome in our external references list, while keeping all other parameters the same. The resulting 'bestsnp.fasta' and 'master.tsv' files are used in RAxML and a tree is produced.  

By rooting the tree with the two B. sp B01 and B02 genomes, we find that the branch lengths are reasonably improved among the B. suis clade, while the two rodent-associated Brucella clades are not well discriminated. The following conclusions were reached:  

1. Our overall tree topology did not change, confirming that the removal of Ochrobactrum was not going to influence our inferences of evolutionary relationships (nor should it). Specifically, the suis-associated samples all continue to cluster in the groups related to B. suis.  

2. Sample 0440 is not clearly associated with one specific subclade among the B. suis clade. To improve the tree topology among those samples a B. suis ONLY set of external reference needs to be run (see below) to improve the relationships among samples and known Brucella suis (and canis) genomes.  

3. All the Australian samples associated with rodent hosts show limited divergence from one another. It is not possible to determien if these are clonal or if meaningful differences exist with such a broad set of Brucella genomes. A series of trees using exclusively rodent-associated Brucella are performed to improve the topology of this clade (see below).

## Targeted B. suis analyses

### Using .vcf as input rather than reads
As explained above, this analysis targeted the suite of samples that associated with B. suis and B. canis references. These samples were: __0436, 0437, 0438, 0439, 0440, and 0450__.  NASP was run once more, however because we are use the same reference sample (B. suis ATCC 23445), the .vcf files were used directly within the pipeline rather than rerunning BWA and GATK from the beginning. The following inputs were utilized:  

> - reference genome = B. suis ATCC 23445
> - external references include all Brucella suis and Brucella canis (n = 55) species avaiable on our servers
> - input are the .vcf files from the first NASP run

To improve the resolution of our tree topology additional external references were added to the "external references" component of the pipeline. All references available in our /leo database on the "pinky" server were included. The output of this pipeline was again a "bestsnp.fasta" and "master.tsv" file that could be analyzed using RAxML once more, with the output of that pipeline being used to create a tree.

### Tree analysis
The resulting tree included clarified the association of our samples with their specific B. suis biovars.  

Of note:  

1. Sample 0440 clustered to the only biovar5 representative, B. suis bv5 str 513.
2. Samples 0438 and 0450 clustered to the biovar 1 clade (note that 0450 was from Australia)
3. Sample 0439 clustered to biovar4? clade (the Canis subtype).
4. Samples 0436 and 0437 clustered to the biovar2.  

This strengthened our earlier claim from the first run - these samples represent a diverse set of biovars among the B. suis clade (really, they are in every subclade including 'canis'). Perhaps of particular importance is the fact that the Australian sample, derived from a pig liver abscess in North Queensland, and typed as Type 3 
****_does this equate to the predicted Biovar it's found in??_*******


## Targeted B. ?rodent? analyses

As described above, a subset of the samples from Australia clustered around the two known Brucella species extracted from rodent hosts: B. suis sp 8313 and B. suis sp. NF 2653 

This analysis was conducted in three separate NASP runs:  

1. Run1 was performed to create an unrooted tree using a unique reference from earlier runs. As such, the pipeline was started at the beginning

### 